This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [5]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'pM_GVkmZMLMPUzwrY83f'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [6]:
# First, import the relevant modules
import requests
import json

In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key=' + API_KEY)
stock_one_day = response.json()

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(stock_one_day['dataset_data'].keys())
print(stock_one_day['dataset_data']['column_names'])
print(stock_one_day['dataset_data']['data'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
# pulling data from API and converting into JSON
response = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)
stock_price_2017 = json.loads(response.content)

In [10]:
# reducing use of lists and list methods where possible to optimize speed & memory usage- not an issue here since it's only one year data
open_price_max = stock_price_2017['dataset_data']['data'][0][1]
open_price_min = open_price_max
intraday_chg = abs(stock_price_2017['dataset_data']['data'][0][2] - stock_price_2017['dataset_data']['data'][0][3])
chg_price = []
volume = []


for idx, date in enumerate(stock_price_2017['dataset_data']['data']):
    open_price_max = open_price_max if (date[1] is None) else (date[1] if date[1] > open_price_max else open_price_max) 
    open_price_min = open_price_min if (date[1] is None) else (date[1] if date[1] < open_price_min else open_price_min) 
    intraday_chg = (date[2] - date[3]) if (date[2] - date[3]) > intraday_chg else intraday_chg
    chg_price.append((stock_price_2017['dataset_data']['data'][idx+1][4] - date[4]) 
                         if (idx+1) < len(stock_price_2017['dataset_data']['data']) else 0)
    volume.append(date[6])  


print('Highest opening price in 2017 was €%.2f' % open_price_max)
print('Lowest opening price in 2017 was €%.2f' % open_price_min)
print('Largest change in price in a day was €%.2f' % intraday_chg)
chg_price = list(map(abs, chg_price))
print('Largest change in price between days was €%.2f' % max(chg_price))
print('Avg trading volume was ' + "{0:,.2f}".format(sum(volume)/len(volume)))
volume.sort()
print('Median trading volume was ' + "{0:,.2f}".format(volume[round(len(volume)/2)]))

Highest opening price in 2017 was €53.11
Lowest opening price in 2017 was €34.00
Largest change in price in a day was €2.81
Largest change in price between days was €2.56
Avg trading volume was 89,124.34
Median trading volume was 76,600.00


In [11]:
# using only list and list methods gives the same answer
open_price = []
intraday_chg = []
close_price = []
volume = []

for date in stock_price_2017['dataset_data']['data']:
    open_price.append(date[1])
    intraday_chg.append(date[2] - date[3])
    close_price.append(date[4])
    volume.append(date[6])  

open_price = [p for p in open_price if p is not None]
print('Highest opening price in 2017 was €%.2f' % max(open_price))
print('Lowest opening price in 2017 was €%.2f' % min(open_price))
print('Largest change in price in a day was €%.2f' % max(list(map(abs, intraday_chg))))

chg_price = list(map(abs, [(p2 - p1) for p1, p2 in zip(close_price[:-1], close_price[1:])]))
print('Largest change in price between days was €%.2f' % max(chg_price))
print('Avg trading volume was ' + "{0:,.2f}".format(sum(volume)/len(volume)))
volume.sort()
print('Median trading volume was ' + "{0:,.2f}".format(volume[round(len(volume)/2)]))

Highest opening price in 2017 was €53.11
Lowest opening price in 2017 was €34.00
Largest change in price in a day was €2.81
Largest change in price between days was €2.56
Avg trading volume was 89,124.34
Median trading volume was 76,600.00
